In [1]:
from alpha_shapes import * 
from astropy import units as u
from astropy.io import fits
from hiresprv.auth import login
from hiresprv.idldriver import Idldriver
from hiresprv.database import Database
from hiresprv.download import Download
from PyAstronomy import pyasl
from specutils.fitting import fit_generic_continuum
from specutils.spectra import Spectrum1D
from scipy import interpolate 

import numpy as np
import pandas as pd 
import os
import sys


In [2]:

#==============
# DO NOT COPY OVER
spectraOutputPath = './SpectraOutput'
rvOutputPath='./RVOutput'
spectraOutputPath = './SpectraOutput'
crossMatchedNames = pd.read_csv("../SPOCSdata/testStars.txt",sep=" ")
star_ID_array = crossMatchedNames["HIRES"].to_numpy()
spectraDic = {} 
# DO NOT COPY OVER
#==============

In [3]:

login('prv.cookies')                                               # For logging into the NExSci servers 
idldriver = Idldriver('prv.cookies')                          # For creating RV scripts 
state = Database('prv.cookies')                               # For retrieving data from HIRES 
dataSpectra = Download('prv.cookies',spectraOutputPath)       # For downloading Spectra
dataRV = Download('prv.cookies', rvOutputPath)                # For downloading RV 
star_ID_array = star_ID_array 





Successful login as jgussman


In [4]:
def calculate_SNR(spectrum):
    spectrum = spectrum[7:10].flatten() # Using echelle orders in the middle 
    SNR = np.mean(np.sqrt(spectrum))
    return SNR 

In [5]:
def DownloadSpectrum(filename):
    '''Download Individual Spectrum and ivar 

    Input: filename (str): name of spectrum you want to download 

    Output: None
    '''
    #print(f"Downloading {filename}")
    dataSpectra.spectrum(filename.replace("r",""))  #Download spectra  
    file_path = os.path.join(dataSpectra.localdir,filename + ".fits")
    temp_deblazedFlux = fits.getdata(file_path)

    return temp_deblazedFlux
    
def DeleteSpectrum(filename):

    #print(f"Deleting {filename}.fits")
    file_path = os.path.join(dataSpectra.localdir,filename + ".fits")
    os.remove(file_path)


In [11]:



hiresName_fileName_snr_dic = {"HIRESName": [],"FILENAME":[],"SNR":[]}  
removed_stars = {"no RV observations":0,"rvcurve wasn't created":0, "SNR < 100":0,"NAN value in spectra":0}
rvDownloadLocation = dataRV.localdir

for name in star_ID_array:
    #Make sure the data is in workspace
    
    try:
        rtn = dataRV.rvcurve(name)
        local_path = os.path.join(rvDownloadLocation, "vst" + name + ".csv")
        temp_df = pd.read_csv(local_path)
        
        if not temp_df.empty:
            filenames = temp_df['FILENAME'].to_numpy()
            
            # Get the spectrum with the largest SNR 
            best_SNR = (np.nan,0,np.nan)
            for filename in filenames:
                temp_spec = DownloadSpectrum(filename)
                temp_SNR = calculate_SNR(temp_spec)
                
                if type(best_SNR[0]) != str:
                    best_SNR = (filename, temp_SNR,temp_spec)
                elif temp_SNR > best_SNR[1]:
                    best_SNR = (filename, temp_SNR,temp_spec)
                    DeleteSpectrum(best_SNR[0])
                else:
                    del temp_spec
                    DeleteSpectrum(filename)
            
            
            best_spectrum = best_SNR[2].flatten()
            if best_SNR[1] < 100:
                removed_stars["SNR < 100"] += 1 
                filename_to_delete = best_SNR[0]
                del best_SNR
                DeleteSpectrum(filename_to_delete)
            elif np.isnan(best_spectrum.flatten()).any():
                removed_stars["NAN value in spectra"] += 1
                filename_to_delete = best_SNR[0]
                del best_SNR
                DeleteSpectrum(filename_to_delete)
            else:
                hiresName_fileName_snr_dic["HIRESName"].append(name)
                hiresName_fileName_snr_dic["FILENAME"].append(best_SNR[0])
                hiresName_fileName_snr_dic["SNR"].append(best_SNR[1])
                del best_SNR
                spectraDic[filename] = best_spectrum

                # ********UNDO**************************
                #SigmaCalculation(best_SNR[0]) #To get the sigma for Inverse variance 
                
        else:
            removed_stars["no RV observations"] +=1 

    except OSError: #This error occurs because for some reason the star's rvcurve wasn't created
            #This is for removing these stars that have no RV metadata 
            removed_stars["rvcurve wasn't created"] += 1
            continue



df = pd.DataFrame(hiresName_fileName_snr_dic) 
filename_snr_df = df.dropna() #If you don't drop the na's then other methods will break
filename_snr_df.to_csv("HIRES_Filename_snr.csv",index_label=False,index=False)
print("Downloading Spectra Has Finished")

In [12]:
hiresName_fileName_snr_dic

{'HIRESName': ['HD1449', 'HD5873', 'HD118744'],
 'FILENAME': ['r20100719.67', 'r20100723.63', 'r20070426.143'],
 'SNR': [149.5636, 109.07611, 132.95981]}

In [13]:
removed_stars

{'no RV observations': 13,
 "rvcurve wasn't created": 5,
 'SNR < 100': 0,
 'NAN value in spectra': 0}

In [20]:
star_ID_array.shape

(21,)

In [14]:
spectraDic

{'r20150204.68': array([  45.8  , 8600.265, 8747.286, ..., 7901.342, 9061.123, 1000.5  ],
       dtype=float32),
 'r20120827.127': array([  114.4   ,  4882.1953,  4798.2246, ..., 10214.058 , 10161.579 ,
         1000.5   ], dtype=float32),
 'r20131214.208': array([   42.8   ,  6954.4604,  6934.334 , ..., 14509.275 , 13029.768 ,
         1000.5   ], dtype=float32)}